In [1]:
import scanpy as sc
import scanpy.external as sce
import matplotlib.pyplot as plt
from matplotlib import rcParams, rc_context
from scToolsGW import percent_cluster_expressing_gene, subsample_by_cluster
import numpy as np
import seaborn as sns
import os
rcParams['svg.fonttype'] = 'none'

os.getcwd()

'/mnt/c/Users/gwakelin/Desktop/johnstonLab_Projects/alasdair_paper/Reanalyzing_Alasdair_Scrublet_07252022gw'

In [2]:
sc.set_figure_params(scanpy=True, frameon=False, format='svg', transparent=True, vector_friendly=False, figsize=[4,4], dpi=80, dpi_save=300, color_map='viridis_r')
sc.logging.print_version_and_date()

Running Scanpy 1.9.1, on 2022-08-09 14:30.


In [94]:
adata = sc.read_h5ad('human_scrublet_processed.h5ad')
adata.shape

(2208, 33538)

In [22]:
# fix weird anndata thing that happens when object is reloaded
adata.uns['log1p']['base'] = None

# find DEGs
sc.tl.rank_genes_groups(adata, groupby='leiden', method='wilcoxon', pts=True)
df = sc.get.rank_genes_groups_df(adata, group=None, pval_cutoff=0.05)

In [23]:
df[df['group']=='10'][df['scores'] > 0].sort_values('scores',ascending=False)

/tmp/ipykernel_15249/3260220380.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df['group']=='10'][df['scores'] > 0].sort_values('scores',ascending=False)


,group,names,scores,logfoldchanges,pvals,pvals_adj,pct_nz_group,pct_nz_reference
14347,10,KRT18,8.069164,10.678263,7.078120e-16,2.373860e-11,0.958333,0.007784
14348,10,STMN1,5.921572,5.368426,3.188788e-09,3.687118e-06,0.750000,0.074176
14349,10,MDK,4.641258,5.366926,3.462942e-06,1.248819e-03,0.583333,0.047619
14350,10,TDO2,4.570112,29.560928,4.874643e-06,1.668222e-03,0.541667,0.000000
14351,10,BASP1,3.923516,4.139868,8.726588e-05,1.773771e-02,0.500000,0.032051
14352,10,CCND1,3.635228,3.519483,2.777345e-04,4.452199e-02,0.500000,0.075092
14353,10,MEST,3.622995,3.073484,2.912114e-04,4.563855e-02,0.500000,0.068681


In [25]:
adata = adata[~adata.obs['leiden'].isin(['10'])]
adata.obs['cell_types'] = adata.obs['leiden'].copy()
# annotated clusters based on expression of known markers
adata.obs['cell_types'].cat.categories = ['EC1','FAP','PC','EC2','SAT','StrPC','T1MC','T2MC','LYM','MYEL']

/tmp/ipykernel_15249/1754903726.py:2: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['cell_types'] = adata.obs['leiden'].copy()


In [27]:
adata.obs['cell_types'].value_counts()

EC1      562
FAP      440
PC       306
EC2      277
SAT      201
StrPC    143
T1MC      75
T2MC      70
LYM       58
MYEL      52
Name: cell_types, dtype: int64

In [ ]:
# save object
adata.write('human_annotated.h5ad',compression='gzip')